In [1]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split ,GridSearchCV
from sklearn.metrics import f1_score,make_scorer
from sklearn.pipeline import Pipeline,FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier

from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

from sqlalchemy import create_engine

In [3]:
import re
from nltk.corpus import stopwords
from sklearn.preprocessing import StandardScaler,Normalizer
from sklearn.svm import SVC

In [4]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to C:\Users\eng
[nltk_data]     basel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\eng
[nltk_data]     basel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
df=pd.read_csv('clean_data.csv')

As we want to build model for predicting the classes of the messages regardless of other information(genre ,id)
we will drop every column except the categories and the message

In [15]:

X=df['message']
Y=df.drop(['message','genre','id'],axis=1)
print(X.shape)
print(Y.shape)

(26028,)
(26028, 35)


In [7]:
def clean(text):
    
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    tokens=word_tokenize(text)
    stemmer= PorterStemmer()
    stop_words=stopwords.words("english")
    stemmed=[stemmer.stem(w) for w in tokens if w not in stop_words]
    return stemmed

In [11]:
pipeline=Pipeline([('counter',CountVectorizer(tokenizer=clean)),
                  ('tfidf',TfidfTransformer()),
                 ('clf',MultiOutputClassifier(SVC()))])

In [16]:
X_train,X_test,Y_train,Y_test =train_test_split(X,Y)

In [9]:
# function to use  as scorer in grid search as it require only one value to validate performance of estimators
#and we have multi class classification
def performance_metric(y_true, y_pred):
    f1_list = []
    for i in range(np.shape(y_pred)[1]):
        f1 = f1_score(np.array(y_true)[:, i], y_pred[:, i])
        f1_list.append(f1)
        
    score = np.median(f1_list)
    return score

In [ ]:
parameters={'clf__estimator__C':[0.1 ],
           'clf__estimator__kernel':['linear'],
           }
f1_scorer=make_scorer(performance_metric)
gd=GridSearchCV(pipeline,parameters,scoring=f1_scorer)
g=gd.fit(X_train,Y_train)